In [1]:
import pandas as pd
from sqlalchemy import create_engine

import pymysql
pymysql.install_as_MySQLdb()

In [2]:
employment_file = "Resources/employment by district.csv"
employment_df = pd.read_csv(employment_file)
employment_df.head()

,State,State Name,Abbrev,115th Cong. Dist,2012 NAICS Code,NAICS Description,Employment,Empl. - data flag,Empl. - noise flag,"1st quarter payroll ($1,000)",1st quarter payroll - noise flag,"Annual payroll ($1,000)",Annual payroll - noise flag,Number of establishments
0,1,Alabama,AL,1,------,Total for all sectors,"236,720",NaN,G,"2,194,826",G,"9,449,314",G,"15,611"
1,1,Alabama,AL,2,------,Total for all sectors,"215,224",NaN,G,"1,951,501",G,"8,266,520",G,"14,252"
2,1,Alabama,AL,3,------,Total for all sectors,"179,100",NaN,G,"1,451,497",G,"6,150,074",G,"11,510"
3,1,Alabama,AL,4,------,Total for all sectors,"188,796",NaN,G,"1,550,243",G,"6,497,936",G,"12,538"
4,1,Alabama,AL,5,------,Total for all sectors,"255,797",NaN,G,"2,732,797",G,"11,465,689",G,"15,137"


In [3]:
employment_df.columns.values

array(['State', 'State Name', 'Abbrev', '115th Cong. Dist',
       '2012 NAICS Code', 'NAICS Description', 'Employment',
       'Empl. - data flag', 'Empl. - noise flag',
       ' 1st quarter payroll ($1,000) ',
       '1st quarter payroll - noise flag', ' Annual payroll ($1,000) ',
       'Annual payroll - noise flag', ' Number of establishments '],
      dtype=object)

In [4]:
employment1_df = employment_df[['Abbrev', '115th Cong. Dist', 'Employment', ' Annual payroll ($1,000) ', ' Number of establishments ']].copy()
employment1_df.head()

,Abbrev,115th Cong. Dist,Employment,"Annual payroll ($1,000)",Number of establishments
0,AL,1,"236,720","9,449,314","15,611"
1,AL,2,"215,224","8,266,520","14,252"
2,AL,3,"179,100","6,150,074","11,510"
3,AL,4,"188,796","6,497,936","12,538"
4,AL,5,"255,797","11,465,689","15,137"


In [5]:
employment1_df = employment1_df.rename(columns={"Abbrev":"state","115th Cong. Dist":"district","Employment":"employment", " Annual payroll ($1,000) ":"annual_payroll_in_1000", " Number of establishments ":"num_of_establishment"})
employment1_df.head()

,state,district,employment,annual_payroll_in_1000,num_of_establishment
0,AL,1,"236,720","9,449,314","15,611"
1,AL,2,"215,224","8,266,520","14,252"
2,AL,3,"179,100","6,150,074","11,510"
3,AL,4,"188,796","6,497,936","12,538"
4,AL,5,"255,797","11,465,689","15,137"


In [6]:
employment1_df.loc[employment1_df.district == 0, 'district'] = "00"
employment1_df.loc[employment1_df.district == 1, 'district'] = "01"
employment1_df.loc[employment1_df.district == 2, 'district'] = "02"
employment1_df.loc[employment1_df.district == 3, 'district'] = "03"
employment1_df.loc[employment1_df.district == 4, 'district'] = "04"
employment1_df.loc[employment1_df.district == 5, 'district'] = "05"
employment1_df.loc[employment1_df.district == 6, 'district'] = "06"
employment1_df.loc[employment1_df.district == 7, 'district'] = "07"
employment1_df.loc[employment1_df.district == 8, 'district'] = "08"
employment1_df.loc[employment1_df.district == 9, 'district'] = "09"


In [7]:
employment1_df.head()

,state,district,employment,annual_payroll_in_1000,num_of_establishment
0,AL,01,"236,720","9,449,314","15,611"
1,AL,02,"215,224","8,266,520","14,252"
2,AL,03,"179,100","6,150,074","11,510"
3,AL,04,"188,796","6,497,936","12,538"
4,AL,05,"255,797","11,465,689","15,137"


In [8]:
state_dist_df = employment1_df['state'] + '_' +  employment1_df['district'].astype(str)
state_dist_df.head()

0    AL_01
1    AL_02
2    AL_03
3    AL_04
4    AL_05
dtype: object

In [9]:
employment1_df.insert(loc=0, column='state_dist', value=state_dist_df)
employment1_df.head()

,state_dist,state,district,employment,annual_payroll_in_1000,num_of_establishment
0,AL_01,AL,01,"236,720","9,449,314","15,611"
1,AL_02,AL,02,"215,224","8,266,520","14,252"
2,AL_03,AL,03,"179,100","6,150,074","11,510"
3,AL_04,AL,04,"188,796","6,497,936","12,538"
4,AL_05,AL,05,"255,797","11,465,689","15,137"


In [10]:
employment1_df['employment'] = employment1_df['employment'].str.replace(',', '')
employment1_df['employment'] = employment1_df['employment'].astype(int)

employment1_df['annual_payroll_in_1000'] = employment1_df['annual_payroll_in_1000'].str.replace(',', '')
employment1_df['annual_payroll_in_1000'] = employment1_df['annual_payroll_in_1000'].astype(int)

employment1_df['num_of_establishment'] = employment1_df['num_of_establishment'].str.replace(',', '')
employment1_df['num_of_establishment'] = employment1_df['num_of_establishment'].astype(int)


employment1_df.head()

,state_dist,state,district,employment,annual_payroll_in_1000,num_of_establishment
0,AL_01,AL,01,236720,9449314,15611
1,AL_02,AL,02,215224,8266520,14252
2,AL_03,AL,03,179100,6150074,11510
3,AL_04,AL,04,188796,6497936,12538
4,AL_05,AL,05,255797,11465689,15137


In [11]:
connection_string = "root:NewPassword@localhost:3306/election_2018_db"
engine = create_engine(f'mysql://{connection_string}')

In [12]:
engine.table_names()

C:\Users\m_ron\Anaconda3\envs\PythonData\lib\site-packages\pymysql\cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, which will be replaced by UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


['candidate_finance',
 'congressional_district_employment',
 'congressional_district_unemployment',
 'house_candidate',
 'senate_candidate']

In [13]:
employment1_df.to_sql(name='congressional_district_employment', con=engine, if_exists='append', index=False)